In [1]:
import pandas as pd

In [12]:
from pymilvus import MilvusClient

milvus_client = MilvusClient("http://localhost:19530")
# milvus_client.drop_collection(collection_name='text')
# milvus_client.drop_collection(collection_name='tables')
# milvus_client.drop_collection(collection_name='charts')
# milvus_client.drop_collection(collection_name='multimodal')

In [ ]:
from nv_ingest_client.client import Ingestor

ingestor = (
    Ingestor(message_client_hostname="localhost")
    .files("data/earnings_consulting/*/*.pdf")
    .extract(
        extract_text=True,
        extract_tables=True,
        extract_charts=True,
        extract_images=True,
        text_depth="page",
    ).embed()
)

results = ingestor.ingest()

In [13]:
len(results)

514

In [ ]:
# Optional: save results
import pickle

filehandler = open('earnings_results.obj', 'wb')
pickle.dump(results, filehandler)

In [9]:
# Optional: load results
import pickle

filehandler = open('earnings_results.obj', 'rb')
results = pickle.load(filehandler)

In [16]:
from nv_ingest_client.util.milvus import write_to_nvingest_collection, create_nvingest_collection, nvingest_retrieval

sparse = False
milvus_hostname = "localhost"
create_nvingest_collection("text", f"http://{milvus_hostname}:19530", sparse=sparse)

/raid/cjarrett/miniconda3/envs/nv-ingest-dev-4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /raid/cjarrett/miniconda3/envs/nv-ingest-
[nltk_data]     dev-4/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [30]:
text_results = [[element for element in results if element['document_type'] == 'text' and element['metadata']['embedding'] is not None] for results in results]

In [31]:
write_to_nvingest_collection(text_results, "text", sparse=sparse, milvus_uri=f"http://{milvus_hostname}:19530", minio_endpoint="localhost:9000")

Wrote data to: [['42ab4bb2-de5f-4764-b6a0-636bef1f5a95/1.parquet']]
Start time: 2025-02-25 15:17:43
Imported row count: 12452
Bulk text upload took 36.10435461997986 s


In [32]:
from collections import defaultdict
from tqdm import tqdm
import os
import numpy as np

def get_recall_scores(query_df, collection_name):
    hits = defaultdict(list)
    all_answers = nvingest_retrieval(
        df_query["query"].to_list(),
        collection_name,
        hybrid=sparse,
        embedding_endpoint="http://localhost:8012/v1",
        model_name="nvidia/llama-3.2-nv-embedqa-1b-v2",
        top_k=10,
    )

    for i in range(len(df_query)):
        expected_pdf_page = query_df['pdf_page'][i]
        retrieved_answers = all_answers[i]
        retrieved_pdfs = [os.path.basename(result['entity']['source']['source_id']).split('.')[0] for result in retrieved_answers]
        retrieved_pages = [str(result['entity']['content_metadata']['page_number']) for result in retrieved_answers]
        retrieved_pdf_pages = [f"{pdf}_{page}" for pdf, page in zip(retrieved_pdfs, retrieved_pages)]    

        for k in [1, 5, 10]:
            hits[k].append(expected_pdf_page in retrieved_pdf_pages[:k])
    
    for k in hits:
        print(f'  - Recall @{k}: {np.mean(hits[k]) :.3f}')

In [33]:
df_query = pd.read_csv('../data/earnings_consulting_multimodal.csv')
df_query['pdf_page'] = df_query.apply(lambda x:f"{x.pdf}_{x.page}", axis=1)
df_query.head()

,dir,pdf,page,query,answer,modality,xywh,pdf_page
0,amazon_earnings_call,004e15aa-5d50-4fb3-9e2f-0ad36639778f,7,How many shares of Rivian’s Class A stock are ...,158 million,text,NaN,004e15aa-5d50-4fb3-9e2f-0ad36639778f_7
1,amazon_earnings_call,004e15aa-5d50-4fb3-9e2f-0ad36639778f,9,What is fair value?,the price that would be received to sell an as...,text,NaN,004e15aa-5d50-4fb3-9e2f-0ad36639778f_9
2,amazon_earnings_call,004e15aa-5d50-4fb3-9e2f-0ad36639778f,33,What are the the key business and industry cha...,"we have many competitors across geographies, i...",text,NaN,004e15aa-5d50-4fb3-9e2f-0ad36639778f_33
3,amazon_earnings_call,06c52e44-38c4-42b0-90c4-14916435e83e,22,What was the percentage increase in Amazon's s...,15%,text,NaN,06c52e44-38c4-42b0-90c4-14916435e83e_22
4,amazon_earnings_call,06c52e44-38c4-42b0-90c4-14916435e83e,30,What risk do we face with foreign exchange rat...,"foreign exchange rates vary, net sales and oth...",text,NaN,06c52e44-38c4-42b0-90c4-14916435e83e_30


In [34]:
df_query.modality.value_counts()

modality
text           242
table          157
chart          129
infographic    100
Name: count, dtype: int64

In [35]:
get_recall_scores(df_query, "text")

  - Recall @1: 0.369
  - Recall @5: 0.533
  - Recall @10: 0.624
